In [1]:
!pip install pygame

In [11]:
import pygame
import random
import time

# Initialize Pygame
pygame.init()

# Constants
WIDTH, HEIGHT = 800, 600
BALL_SIZE = 20
PADDLE_WIDTH, PADDLE_HEIGHT = 10, 100
PADDLE_SPEED = 7
BALL_SPEED_X, BALL_SPEED_Y = 5, 5
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)

# Set up the display
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Pong")

# Game objects
ball = pygame.Rect(WIDTH // 2 - BALL_SIZE // 2, HEIGHT // 2 - BALL_SIZE // 2, BALL_SIZE, BALL_SIZE)
left_paddle = pygame.Rect(50, HEIGHT // 2 - PADDLE_HEIGHT // 2, PADDLE_WIDTH, PADDLE_HEIGHT)
right_paddle = pygame.Rect(WIDTH - 50 - PADDLE_WIDTH, HEIGHT // 2 - PADDLE_HEIGHT // 2, PADDLE_WIDTH, PADDLE_HEIGHT)

ball_speed_x = BALL_SPEED_X * random.choice((1, -1))
ball_speed_y = BALL_SPEED_Y * random.choice((1, -1))

AI_score, player_score = 0, 0
font = pygame.font.Font(None, 74)

# AI base parameters
AI_BASE_SPEED = 12
AI_MAX_SPEED = 16
AI_MIN_SPEED = 8
AI_REACTION_DELAY_BASE = 1
AI_REACTION_DELAY_MIN = 0
AI_REACTION_DELAY_MAX = 2

# Game loop
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            pygame.display.quit()
            pygame.quit()
            break

    # Adjust AI difficulty based on score difference from the player
    score_diff = player_score - AI_score
    ai_speed = AI_BASE_SPEED + score_diff
    ai_speed = max(AI_MIN_SPEED, min(AI_MAX_SPEED, ai_speed))
    
    ai_reaction_delay = AI_REACTION_DELAY_BASE - score_diff
    ai_reaction_delay = max(AI_REACTION_DELAY_MIN, min(AI_REACTION_DELAY_MAX, ai_reaction_delay))

    # Right paddle (player) movement
    keys = pygame.key.get_pressed()
    if keys[pygame.K_UP] and right_paddle.top > 0:
        right_paddle.y -= PADDLE_SPEED
    if keys[pygame.K_DOWN] and right_paddle.bottom < HEIGHT:
        right_paddle.y += PADDLE_SPEED

    # Left paddle (AI) movement with dynamic difficulty
    if ball.centery > left_paddle.centery and left_paddle.bottom < HEIGHT:
        if random.randint(0, ai_reaction_delay) == 0:
            left_paddle.y += ai_speed
    elif ball.centery < left_paddle.centery and left_paddle.top > 0:
        if random.randint(0, ai_reaction_delay) == 0:
            left_paddle.y -= ai_speed

    # Ball movement
    ball.x += ball_speed_x
    ball.y += ball_speed_y

    # Collision with top/bottom walls
    if ball.top <= 0 or ball.bottom >= HEIGHT:
        ball_speed_y *= -1

    # Collision with paddles
    if ball.colliderect(left_paddle) or ball.colliderect(right_paddle):
        ball_speed_x *= -1

    # Ball out of bounds
    if ball.left <= 0:
        player_score += 1
        ball = pygame.Rect(WIDTH // 2 - BALL_SIZE // 2, HEIGHT // 2 - BALL_SIZE // 2, BALL_SIZE, BALL_SIZE)
        ball_speed_x = BALL_SPEED_X * random.choice((1, -1))
        ball_speed_y = BALL_SPEED_Y * random.choice((1, -1))

    if ball.right >= WIDTH:
        AI_score += 1
        ball = pygame.Rect(WIDTH // 2 - BALL_SIZE // 2, HEIGHT // 2 - BALL_SIZE // 2, BALL_SIZE, BALL_SIZE)
        ball_speed_x = BALL_SPEED_X * random.choice((1, -1))
        ball_speed_y = BALL_SPEED_Y * random.choice((1, -1))

    # Drawing
    screen.fill(BLACK)
    pygame.draw.rect(screen, WHITE, left_paddle)
    pygame.draw.rect(screen, WHITE, right_paddle)
    pygame.draw.ellipse(screen, WHITE, ball)
    pygame.draw.aaline(screen, WHITE, (WIDTH // 2, 0), (WIDTH // 2, HEIGHT))

    left_text = font.render(str(AI_score), True, WHITE)
    screen.blit(left_text, (WIDTH // 4, 20))
    right_text = font.render(str(player_score), True, WHITE)
    screen.blit(right_text, (WIDTH * 3 // 4, 20))

    pygame.display.flip()
    pygame.time.Clock().tick(60)

    time.sleep(0.01)

pygame.quit()


error: video system not initialized